In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("DAG and Broadcast Demo").getOrCreate

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DAG and Broadcast Demo").getOrCreate() # Correctly initialize spark

orders_data = [
    ("O001","Hyderabad",1200),
    ("O002","Delhi",800),
    ("O003","Mumbai",1500),
    ("O004","Bangalore",400),
    ("O005","Hyderabad",300),
    ("O006","Delhi",2000),
    ("O007","Mumbai",700),
    ("O008","Bangalore",1800),
    ("O009","Delhi",350),
    ("O010","Hyderabad",900)
]

orders_cols = ["order_id","city","order_amount"]

orders_df = spark.createDataFrame(orders_data, orders_cols)
orders_df.show()

+--------+---------+------------+
|order_id|     city|order_amount|
+--------+---------+------------+
|    O001|Hyderabad|        1200|
|    O002|    Delhi|         800|
|    O003|   Mumbai|        1500|
|    O004|Bangalore|         400|
|    O005|Hyderabad|         300|
|    O006|    Delhi|        2000|
|    O007|   Mumbai|         700|
|    O008|Bangalore|        1800|
|    O009|    Delhi|         350|
|    O010|Hyderabad|         900|
+--------+---------+------------+



In [ ]:
city_data = [
    ("Hyderabad","Tier-1"),
    ("Delhi","Tier-1"),
    ("Mumbai","Tier-1"),
    ("Bangalore","Tier-1")
]

city_cols = ["city","city_category"]

city_df = spark.createDataFrame(city_data, city_cols)
city_df.show()

+---------+-------------+
|     city|city_category|
+---------+-------------+
|Hyderabad|       Tier-1|
|    Delhi|       Tier-1|
|   Mumbai|       Tier-1|
|Bangalore|       Tier-1|
+---------+-------------+



In [ ]:
filtered_orders = orders_df.filter(col("order_amount") > 500)

In [ ]:
joined_df = filtered_orders.join(
    city_df,
    on="city",
    how="inner"
)

final_df = joined_df.select(
    "order_id",
    "city",
    "city_category",
    "order_amount"
)

In [ ]:
final_df.explain(True)

== Parsed Logical Plan ==
'Project ['order_id, 'city, 'city_category, 'order_amount]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- LogicalRDD [city#13, city_category#14], false

== Analyzed Logical Plan ==
order_id: string, city: string, city_category: string, order_amount: bigint
Project [order_id#0, city#1, city_category#14, order_amount#2L]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- LogicalRDD [city#13, city_category#14], false

== Optimized Logical Plan ==
Project [order_id#0, city#1, city_category#14, order_amount#2L]
+- Join Inner, (city#1 = city#13)
   :- Filter ((isnotnull(order_amount#2L) AND (order

In [ ]:
from pyspark.sql.functions import broadcast

broadcast_join_df = filtered_orders.join(
    broadcast(city_df),
    on="city",
    how="inner"
)

final_boradcast_df = broadcast_join_df.select(
    "order_id",
    "city",
    "city_category",
    "order_amount"
)

In [ ]:
final_boradcast_df.explain(True)

== Parsed Logical Plan ==
'Project ['order_id, 'city, 'city_category, 'order_amount]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [city#13, city_category#14], false

== Analyzed Logical Plan ==
order_id: string, city: string, city_category: string, order_amount: bigint
Project [order_id#0, city#1, city_category#14, order_amount#2L]
+- Project [city#1, order_id#0, order_amount#2L, city_category#14]
   +- Join Inner, (city#1 = city#13)
      :- Filter (order_amount#2L > cast(500 as bigint))
      :  +- LogicalRDD [order_id#0, city#1, order_amount#2L], false
      +- ResolvedHint (strategy=broadcast)
         +- LogicalRDD [city#13, city_category#14], false

== Optimized Logical Plan ==
Project [order_id#0, city#1, city_category#14, order_amoun